In [1]:
%load_ext autoreload
%autoreload 2
import glob
import extract_text
import gensim
import numpy as np
import text_preprocessing
from tqdm import tqdm_notebook as tqdm
import util

/home/charles/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
#open the files
all_files = glob.glob('files/*/*')

In [ ]:
all_text = []
for f in tqdm(all_files):
    try:
        all_text.append(extract_text.apply(f))
    except:
        print("passed")
        pass
util.save(all_text, 'safe/all_text.p')

In [2]:
#load preread text (speedup)
all_text = util.load('safe/all_text.p')

In [8]:
preprocessor = text_preprocessing.Preprocessor(max_nb_words=100000)
%time sentenses, dico = preprocessor.fit_and_vectorize(all_text)
util.save(dico, 'safe/dico_copus.p')
util.save(sentenses, 'safe/sentenses.p')

100000


100%|██████████| 375/375 [09:26<00:00,  1.51s/it]


CPU times: user 38min 47s, sys: 52min 24s, total: 1h 31min 11s
Wall time: 9min 36s


In [10]:
dico = util.load('safe/dico_copus.p')
sentenses = util.load('safe/sentenses.p')

In [12]:
dico_revers = {dico[k]:k for k in dico}

In [13]:
sentenses_word = []
for sent in sentenses:
    sent = [dico_revers[i] for i in sent]
    sentenses_word.append(sent)

In [14]:
%time model = gensim.models.Word2Vec(sentences=sentenses_word, min_count=1, iter=10, size=300, max_vocab_size=100000)# default size is 100

CPU times: user 1min 3s, sys: 235 ms, total: 1min 4s
Wall time: 24.2 s


In [15]:
#https://codekansas.github.io/blog/2016/gensim.html
vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
weights = model.wv.vectors
np.save(open('safe/embeddings.np', 'wb'), weights)
util.save(vocab, 'safe/vocab_gensim.p')

99999


In [18]:
model.wv.most_similar('prudential')#(str(dico['prudential']))

[('nintendo', 0.5323249101638794),
 ('sustainability', 0.5225441455841064),
 ('prei', 0.5014888048171997),
 ('commitment', 0.495370090007782),
 ('tangram', 0.4861685633659363),
 ('inclusion', 0.4757257103919983),
 ('loom', 0.475308895111084),
 ('morningstar', 0.47198575735092163),
 ('ecosystems', 0.47135046124458313),
 ('smokefree', 0.47037386894226074)]

In [21]:
model.wv.similarity('prudential', 'company')

0.4147244019999571